# Project 3: Battery Equivalent Circuit Model Design
>**Date Created**: 19 February 2024<br>
>**Course**: AEM566 - Optimal Estimation<br>
>**Author**: Benjamin Lucke<br>

In [396]:
import numpy as np
import scipy as sp
import pandas as pd
from matplotlib import pyplot as plt
from scipy.integrate import trapezoid, simpson

### 1. Load the battery data for the pulse discharge tests in the accompanying CSV file

In [567]:
data = pd.read_csv('data/pulse_discharge_test_data.csv')
# rename columns to make easier to reference and convert times to datetime for convenience
columnMapper = dict(zip(data.columns.values,['t', 'v', 'i'],)) 
data = data.rename(columns=columnMapper)
data['t'] = pd.to_datetime(data['t'],format="%H:%M:%S")

### 2. Numerically integrate the current to obtain the SOC for the entire data set

In [574]:
soc_full = simpson(data['i'], dx = 10)

### 3. Partition the data into each rest period using the current measurement as the trigger

In [569]:
data['i_diff'] = data.loc[:,'i'].diff()
delta_indices = data[(data['i_diff']) > .1].index.tolist() # Get indices based on differnetial in current
pairs = list()

# Create grouped pairs of indices where rest period starts and ends
for k, index in enumerate(delta_indices):
    if k == len(delta_indices)-1:
        pair = index + 118, data.index[-1]
    else:
        pair = index + 118, delta_indices[k+1]
    pairs.append(pair)
pairs = np.array(pairs)

In [570]:
def estimate_r0(rest_start, rest_end):
    v_ts = data.iloc[rest_start - 1,1] # get voltage at time step before rest period starts 
    v_tr = data.iloc[rest_start, 1] # get voltage at start of rest period
    rhs = v_tr - v_ts # Compute difference in voltages
    r0 = rhs/data.iloc[rest_start,2] # divide by current to get estimate of R0
    return r0

In [575]:
def estimate_r1(end):
    ocv = data.iloc[end, 1]
    print(ocv)